This is an adaptation of the **RFdiffusion** Colab Implementation to run locally in Jupyter Notebook, preferably on the LRZ AI Cluster. It does not require any prior installations, you just have to keep in mind that all data is going to be saved in your home directory in a new directory called "RFdiffusionJupyter". So if you have RFdiffusion installed already you could for example save a little disk space by combining this directory with the original RFdiffusion directory. 
The original Colab Notebook can be found here:
https://colab.research.google.com/github/sokrypton/ColabDesign/blob/main/rf/examples/diffusion_ori.ipynb

First thing to do is the installation and definition of functions. Don´t worry, if you already have installed it should skip installation. If you change any path names, etc. in your UI be sure to adapt in the code below! There might be problems with the PyTorch versions, so if all fails, just delete the directory RFdiffusionJupyter (after saving all your previous in and outputs as necessary!!!!) and run the installation cell again! 

In [ ]:
import os, subprocess, sys, shutil, time, random, string

start = time.time()

# Function to run commands and hide output
def run_command(command):
    subprocess.run(command, shell=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

# Clone RFdiffusion repository directly into RFdiffusionJupyter
if not os.path.isdir("RFdiffusionJupyter/RFdiffusion"):
    print("Cloning RFdiffusion...")
    os.chdir(os.path.expanduser("~"))
    os.makedirs("RFdiffusionJupyter", exist_ok=True)
    os.chdir(os.path.expanduser("RFdiffusionJupyter"))
    run_command("git clone https://github.com/sokrypton/RFdiffusion.git")
    os.chdir("RFdiffusion")
    run_command("wget -qnc https://files.ipd.uw.edu/krypton/ananas")
    run_command("chmod +x ananas")
    os.chdir(os.path.expanduser("~"))
    print("Cloning RFdiffusion and installing dependencies... Done")

# Install necessary packages
os.chdir(os.path.expanduser("~"))
run_command("pip install ipywidgets py3Dmol")
run_command("pip install jedi omegaconf hydra-core icecream pyrsistent pynvml decorator")
run_command("pip install git+https://github.com/NVIDIA/dllogger#egg=dllogger")
run_command("pip install --no-dependencies dgl==2.0.0 -f https://data.dgl.ai/wheels/cu121/repo.html")
run_command("pip install --no-dependencies e3nn==0.3.3 opt_einsum_fx")
run_command("pip install colabdesign")
run_command("pip install -U 'jax[cuda12]'")
os.chdir(os.path.expanduser("RFdiffusion"))
os.chdir("env/SE3Transformer")
run_command("pip install .")
os.chdir(os.path.expanduser("~"))

# Download model parameters and unzip if necessary
# Check if params are already downloaded
if not os.path.isfile("RFdiffusionJupyter/RFdiffusion/params/done.txt"):
    print("Downloading AF2 params...")
    os.chdir(os.path.expanduser("~"))
    params_dir = "RFdiffusionJupyter/RFdiffusion/params"
    os.makedirs(params_dir, exist_ok=True)
    run_command(f"wget -c https://files.ipd.uw.edu/krypton/schedules.zip -P {params_dir}")
    run_command(f"wget -c https://storage.googleapis.com/alphafold/alphafold_params_2022-12-06.tar -P {params_dir}")
    run_command(f"tar -xf {params_dir}/alphafold_params_2022-12-06.tar -C {params_dir}")
    run_command(f"unzip {params_dir}/schedules.zip -d {params_dir}")
    run_command(f"touch {params_dir}/done.txt")


if not os.path.isdir("RFdiffusionJupyter/RFdiffusion/params/ColabDesign"):
    print("Cloning ColabDesign ...")
    os.chdir(os.path.expanduser("~/RFdiffusionJupyter/RFdiffusion/params"))
    run_command("git clone https://github.com/sokrypton/ColabDesign.git")
    os.chdir(os.path.expanduser("~"))
    print("Cloning ColabDesign and installing dependencies... Done")

os.chdir(os.path.expanduser("~"))

# Check if all models are downloaded into the new /models directory
model_files = ["Base_ckpt.pt", "Complex_base_ckpt.pt", "Complex_beta_ckpt.pt"]
if not all(os.path.isfile(f"RFdiffusionJupyter/RFdiffusion/models/{model}") for model in model_files):
    print("Downloading model weights...")
    models_dir = "RFdiffusionJupyter/RFdiffusion/models"
    run_command(f"wget -c http://files.ipd.uw.edu/pub/RFdiffusion/6f5902ac237024bdd0c176cb93063dc4/Base_ckpt.pt -P {models_dir}")
    run_command(f"wget -c http://files.ipd.uw.edu/pub/RFdiffusion/e29311f6f1bf1af907f9ef9f44b8328b/Complex_base_ckpt.pt -P {models_dir}")
    run_command(f"wget -c http://files.ipd.uw.edu/pub/RFdiffusion/f572d396fae9206628714fb2ce00f72e/Complex_beta_ckpt.pt -P {models_dir}")  
    
if 'RFdiffusionJupyter/RFdiffusion' not in sys.path:
    os.environ["DGLBACKEND"] = "pytorch"
    sys.path.append('RFdiffusionJupyter/RFdiffusion')

os.chdir(os.path.expanduser("RFdiffusionJupyter/RFdiffusion"))

# Import modules
import json
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, HTML
import ipywidgets as widgets
import py3Dmol

# Importing the necessary modules for inference
from inference.utils import parse_pdb
from colabdesign.rf.utils import get_ca, fix_contigs, fix_partial_contigs, fix_pdb, sym_it
from colabdesign.shared.protein import pdb_to_string
from colabdesign.shared.plot import plot_pseudo_3D

def get_pdb(pdb_code=None):
    if pdb_code is None or pdb_code == "":
        raise ValueError("Error: A PDB ID or a path to a PDB file must be provided.")
    if len(pdb_code) == 4:
        if not os.path.isfile(f"{pdb_code}.pdb1"):
            subprocess.run(["wget", "-qnc", f"https://files.rcsb.org/download/{pdb_code}.pdb1.gz"])
            subprocess.run(["gunzip", f"{pdb_code}.pdb1.gz"])
        return f"{pdb_code}.pdb1"
    elif len(pdb_code) == 6 or len(pdb_code) == 10:
        subprocess.run(["wget", "-qnc", f"https://alphafold.ebi.ac.uk/files/AF-{pdb_code}-F1-model_v3.pdb"])
        return f"AF-{pdb_code}-F1-model_v3.pdb"
    elif os.path.isfile(pdb_code):
        return pdb_code
    
def run_ananas(pdb_str, path, sym=None):
    pdb_filename = f"RFdiffusionJupyter/RFdiffusion/outputs/{path}/ananas_input.pdb"
    out_filename = f"RFdiffusionJupyter/RFdiffusion/outputs/{path}/ananas.json"
    with open(pdb_filename, "w") as handle:
        handle.write(pdb_str)

    cmd = f"./ananas {pdb_filename} -u -j {out_filename}"
    if sym is None:
        os.system(cmd)
    else:
        os.system(f"{cmd} {sym}")

    # parse results
    try:
        out = json.loads(open(out_filename, "r").read())
        results, AU = out[0], out[-1]["AU"]
        group = AU["group"]
        chains = AU["chain names"]
        rmsd = results["Average_RMSD"]
        print(f"AnAnaS detected {group} symmetry at RMSD:{rmsd:.3}")

        C = np.array(results['transforms'][0]['CENTER'])
        A = [np.array(t["AXIS"]) for t in results['transforms']]

        # apply symmetry and filter to the asymmetric unit
        new_lines = []
        for line in pdb_str.split("\n"):
            if line.startswith("ATOM"):
                chain = line[21:22]
                if chain in chains:
                    x = np.array([float(line[i:(i+8)]) for i in [30, 38, 46]])
                    if group[0] == "c":
                        x = sym_it(x, C, A[0])
                    if group[0] == "d":
                        x = sym_it(x, C, A[1], A[0])
                    coord_str = "".join(["{:8.3f}".format(a) for a in x])
                    new_lines.append(line[:30] + coord_str + line[54:])
            else:
                new_lines.append(line)
        return results, "\n".join(new_lines)

    except:
        return None, pdb_str

def run(command, steps, num_designs=1, visual="none"):

    def run_command_and_get_pid(command):
        pid_file = '/dev/shm/pid'
        os.system(f'nohup {command} & echo $! > {pid_file}')
        with open(pid_file, 'r') as f:
            pid = int(f.read().strip())
        os.remove(pid_file)
        return pid

    def is_process_running(pid):
        try:
            os.kill(pid, 0)
        except OSError:
            return False
        else:
            return True

    run_output = widgets.Output()
    progress = widgets.FloatProgress(min=0, max=1, description='running', bar_style='info')
    display(widgets.VBox([progress, run_output]))

    # clear previous run
    for n in range(steps):
        if os.path.isfile(f"/dev/shm/{n}.pdb"):
            os.remove(f"/dev/shm/{n}.pdb")

    pid = run_command_and_get_pid(command)
    try:
        fail = False
        for _ in range(num_designs):

            # for each step check if output generated
            for n in range(steps):
                wait = True
                while wait and not fail:
                    time.sleep(0.1)
                    if os.path.isfile(f"/dev/shm/{n}.pdb"):
                        pdb_str = open(f"/dev/shm/{n}.pdb").read()
                        if pdb_str[-3:] == "TER":
                            wait = False
                        elif not is_process_running(pid):
                            fail = True
                    elif not is_process_running(pid):
                        fail = True

                if fail:
                    progress.bar_style = 'danger'
                    progress.description = "failed"
                    break

                else:
                    progress.value = (n + 1) / steps
                if os.path.exists(f"/dev/shm/{n}.pdb"):
                    os.remove(f"/dev/shm/{n}.pdb")
            if fail:
                progress.bar_style = 'danger'
                progress.description = "failed"
                break

        while is_process_running(pid):
            time.sleep(0.1)

    except KeyboardInterrupt:
        os.kill(pid, signal.SIGTERM)
        progress.bar_style = 'danger'
        progress.description = "stopped"

def run_diffusion(contigs, path, pdb=None, iterations=50,
                  symmetry="none", order=1, hotspot=None,
                  chains=None, add_potential=False, partial_T="auto",
                  num_designs=1, use_beta_model=False, visual="none"):

    os.chdir(os.path.expanduser("~"))
    full_path = f"RFdiffusionJupyter/RFdiffusion/outputs/{path}"
    os.makedirs(full_path, exist_ok=True)
    assert os.path.exists(full_path), "Output directory was not created!"
    opts = [f"inference.output_prefix={full_path}",
            f"inference.num_designs={num_designs}"]

    if chains == "":
        chains = None

    # determine symmetry type
    if symmetry in ["auto", "cyclic", "dihedral"]:
        if symmetry == "auto":
            sym, copies = None, 1
        else:
            sym, copies = {"cyclic": (f"c{order}", order),
                           "dihedral": (f"d{order}", order * 2)}[symmetry]
    else:
        symmetry = None
        sym, copies = None, 1

    # determine mode
    contigs = contigs.replace(",", " ").replace(":", " ").split()
    is_fixed, is_free = False, False
    fixed_chains = []
    for contig in contigs:
        for x in contig.split("/"):
            a = x.split("-")[0]
            if a[0].isalpha():
                is_fixed = True
                if a[0] not in fixed_chains:
                    fixed_chains.append(a[0])
            if a.isnumeric():
                is_free = True
    if len(contigs) == 0 or not is_free:
        mode = "partial"
    elif is_fixed:
        mode = "fixed"
    else:
        mode = "free"

    # fix input contigs
    if mode in ["partial", "fixed"]:
        pdb_str = pdb_to_string(get_pdb(pdb), chains=chains)
        if symmetry == "auto":
            a, pdb_str = run_ananas(pdb_str, path)
            if a is None:
                print(f'ERROR: no symmetry detected')
                symmetry = None
                sym, copies = None, 1
            else:
                if a["group"][0] == "c":
                    symmetry = "cyclic"
                    sym, copies = a["group"], int(a["group"][1:])
                elif a["group"][0] == "d":
                    symmetry = "dihedral"
                    sym, copies = a["group"], 2 * int(a["group"][1:])
                else:
                    print(f'ERROR: the detected symmetry ({a["group"]}) not currently supported')
                    symmetry = None
                    sym, copies = None, 1

        elif mode == "fixed":
            pdb_str = pdb_to_string(pdb_str, chains=fixed_chains)

        pdb_filename = f"{full_path}/input.pdb"
        with open(pdb_filename, "w") as handle:
            handle.write(pdb_str)

        parsed_pdb = parse_pdb(pdb_filename)
        opts.append(f"inference.input_pdb={pdb_filename}")
        if mode in ["partial"]:
            if partial_T == "auto":
                iterations = int(80 * (iterations / 200))
            else:
                iterations = int(partial_T)
            opts.append(f"diffuser.partial_T={iterations}")
            contigs = fix_partial_contigs(contigs, parsed_pdb)
        else:
            opts.append(f"diffuser.T={iterations}")
            contigs = fix_contigs(contigs, parsed_pdb)
    else:
        opts.append(f"diffuser.T={iterations}")
        parsed_pdb = None
        contigs = fix_contigs(contigs, parsed_pdb)

    if hotspot is not None and hotspot != "":
        hotspot = ",".join(hotspot.replace(",", " ").split())
        opts.append(f"ppi.hotspot_res='[{hotspot}]'")

    # setup symmetry
    if sym is not None:
        sym_opts = ["--config-name symmetry", f"inference.symmetry={sym}"]
        if add_potential:
            sym_opts += ["'potentials.guiding_potentials=[\"type:olig_contacts,weight_intra:1,weight_inter:0.1\"]'",
                         "potentials.olig_intra_all=True", "potentials.olig_inter_all=True",
                         "potentials.guide_scale=2", "potentials.guide_decay=quadratic"]
        opts = sym_opts + opts
        contigs = sum([contigs] * copies, [])

    opts.append(f"'contigmap.contigs=[{' '.join(contigs)}]'")
    opts += ["inference.dump_pdb=True", "inference.dump_pdb_path='/dev/shm'"]
    if use_beta_model:
        opts += ["inference.ckpt_override_path=RFdiffusionJupyter/Rfdiffusion/models/Complex_beta_ckpt.pt"]

    print("mode:", mode)
    print("output:", full_path)
    print("contigs:", contigs)

    opts_str = " ".join(opts)
    cmd = f"RFdiffusionJupyter/RFdiffusion/run_inference.py {opts_str}"
    print(cmd)

    # RUN
    cmd = f"RFdiffusionJupyter/RFdiffusion/run_inference.py {opts_str}"
    print(f"Running command: {cmd}")
    run(cmd, iterations, num_designs, visual=visual)

    # fix pdbs
    for n in range(num_designs):
        pdbs = [f"RFdiffusionJupyter/RFdiffusion/outputs/traj/{path}_{n}_pX0_traj.pdb",
                f"RFdiffusionJupyter/RFdiffusion/outputs/traj/{path}_{n}_Xt-1_traj.pdb",
                f"{full_path}_{n}.pdb"]
        for pdb in pdbs:
            if not os.path.exists(pdb):
                print(f"WARNING: {pdb} was not generated")
            with open(pdb, "r") as handle:
                pdb_str = handle.read()
            with open(pdb, "w") as handle:
                handle.write(fix_pdb(pdb_str, contigs))
    os.chdir(os.path.expanduser("~"))
    return contigs, copies

end = time.time()
os.chdir(os.path.expanduser("~"))
print(f"Installation took {end-start} seconds")

The following step is actual diffusion. Parameters you can change are:

**name** - name of output files

**contigs** - specify what to diffuse
 - random monomer: contigs = "x" with x the length of amino acid chain
 - binder: specify chain(s) and the relevant amino acids you want to diffuse a binder against as well as the length of the binders amino acid chain; for binding vs a monomer, contigs = "Asss-eee:x" with sss as starting amino acid on chain A, eee ending amino acid on chain A and x the length of the binders amino acid chain; for binding vs a multimer contigs="Csss-eee:C2sss2-eee2:C3sss3-eee3:x" for different chains C and starting/ending amino acid respectively
 - for motif scaffolding or partial diffusion check the original notebook or https://github.com/RosettaCommons/RFdiffusion for instructions

**pdb** - PDB ID of the protein you want to design a binder against, if you use the UNIPROT ID you will design vs the AF prediction from https://alphafold-ebi-ac-uk.translate.goog/?_x_tr_sl=en&_x_tr_tl=de&_x_tr_hl=de&_x_tr_pto=sc&_x_tr_hist=true; If you do have a custom PDB file you can also use by specifying the full file path like "~/RFdiffusion/inputs/someprotein.pdb"

**iterations** ["25", "50", "100", "150", "200"] - number of iterations for the diffusion of a single backbone, increases runtime but smaller more exact steps towards minimum

**hotspot** - preferably hydrophobic amino acids on the specified contig which could increase avidity, as binding is dominated by hydrophobic interactions; specified via hotspot = "Caaa, ..." with chain C and the number of the hotspot residue aaa; 3-6 amino acids work best but exact amount should be experimented

**num_designs** ["1", "2", "4", "8", "16", "32"] - number of diffused designs, list of possible numbers can be neglected, just specify a different number. 

**visual**  - removed as there are some problems within Jupyter Lab, keep it at "none" otherwise it will break

**symmetry** ["none", "auto", "cyclic", "dihedral"] - symmetry of diffused protein

**order** ["1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11", "12"] - order of chosen symmetry

**chains** - filter input PDB to specified chains

**add_potential** ["True", "False"] - discourage clashes between chains

**partial_T** - only used in partial diffusion protocol, check original notebook

**use_beta_model** ["True", "False"] - can be used if you are seeing lots of helices for better SSE balance, however not tested experimentally

In [ ]:
start = time.time()
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
name = "SpyTagScaffold" 
contigs = "B:100" 
pdb = "4MLI" 
iterations = 50 
hotspot = "" 
num_designs = 30
visual = "none" 
symmetry = "cyclic"
order = 3 
chains = "" 
add_potential = True 
partial_T = "auto" 
use_beta_model = False 
# determine where to save
path = name
os.chdir(os.path.expanduser("~"))
while os.path.exists(f"RFdiffusionJupyter/RFdiffusion/outputs/{path}_0.pdb"):
  path = name + "_" + ''.join(random.choices(string.ascii_lowercase + string.digits, k=5))

flags = {"contigs":contigs,
         "pdb":pdb,
         "order":order,
         "iterations":iterations,
         "symmetry":symmetry,
         "hotspot":hotspot,
         "path":path,
         "chains":chains,
         "add_potential":add_potential,
         "num_designs":num_designs,
         "use_beta_model":use_beta_model,
         "visual":visual,
         "partial_T":partial_T}

for k,v in flags.items():
  if isinstance(v,str):
    flags[k] = v.replace("'","").replace('"','')
    
os.chdir(os.path.expanduser("~"))
contigs, copies = run_diffusion(**flags)
end = time.time()
print(f"Diffusion took {end-start} seconds")

 The following code allows you to display your diffused backbones with the specified contig in three dimensions. Here you can specify three parameters:
 
 **animate** - ["none", "movie", "interactive"]
 
 **color** - ["rainbow", "chain", "plddt"]
 
 **dpi** - ["100", "200", "400"]
 
 **design** - specify the design you want to model, starting from 0 which is also the default and will be displayed if you do not specify anthing!

In [ ]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
animate = "none" 
color = "chain"
denoise = True
dpi = 100
design = 2
from colabdesign.shared.plot import pymol_color_list
from colabdesign.rf.utils import get_ca, get_Ls, make_animation
from string import ascii_uppercase,ascii_lowercase
alphabet_list = list(ascii_uppercase+ascii_lowercase)

def plot_pdb(num=0):
  os.chdir(os.path.expanduser("~"))  
  if denoise:
    pdb_traj = f"RFdiffusionJupyter/RFdiffusion/outputs/traj/{path}_{num}_pX0_traj.pdb"
  else:
    pdb_traj = f"RFdiffusionJupyter/RFdiffusion/outputs/traj/{path}_{num}_Xt-1_traj.pdb"
  if animate in ["none","interactive"]:
    hbondCutoff = 4.0
    view = py3Dmol.view(js='https://3dmol.org/build/3Dmol.js')
    if animate == "interactive":
      pdb_str = open(pdb_traj,'r').read()
      view.addModelsAsFrames(pdb_str,'pdb',{'hbondCutoff':hbondCutoff})
    else:
      pdb = f"RFdiffusionJupyter/RFdiffusion/outputs/{path}_{num}.pdb"
      pdb_str = open(pdb,'r').read()
      view.addModel(pdb_str,'pdb',{'hbondCutoff':hbondCutoff})
    if color == "rainbow":
      view.setStyle({'cartoon': {'color':'spectrum'}})
    elif color == "chain":
      for n,chain,c in zip(range(len(contigs)),
                              alphabet_list,
                              pymol_color_list):
          view.setStyle({'chain':chain},{'cartoon': {'color':c}})
    else:
      view.setStyle({'cartoon': {'colorscheme': {'prop':'b','gradient': 'roygb','min':0.5,'max':0.9}}})
    view.zoomTo()
    if animate == "interactive":
      view.animate({'loop': 'backAndForth'})
    view.show()
  else:
    Ls = get_Ls(contigs)
    xyz, bfact = get_ca(pdb_traj, get_bfact=True)
    xyz = xyz.reshape((-1,sum(Ls),3))[::-1]
    bfact = bfact.reshape((-1,sum(Ls)))[::-1]
    if color == "chain":
      display(HTML(make_animation(xyz, Ls=Ls, dpi=dpi, ref=-1)))
    elif color == "rainbow":
      display(HTML(make_animation(xyz, dpi=dpi, ref=-1)))
    else:
      display(HTML(make_animation(xyz, plddt=bfact*100, dpi=dpi, ref=-1)))


if num_designs > 1:
    plot_pdb(design)
else:
  plot_pdb()

The final step is generating a sequence for the diffused backbone with **ProteinMPNN**, model it with **AF2** and evaluate the designs depending on interaction predicted Aligned Error and complex root mean square deviation (i_pAE and rmsd --> best case smaller than ten)! Here you have a number of parameters which you can change: 

***ProteinMPNN settings:***

**num_seqs** ["1", "2", "4", "8", "16", "32", "64"] - number of designed sequences per diffused backbone

**mpnn_sampling_temp** ["0.0001", "0.1", "0.15", "0.2", "0.25", "0.3", "0.5", "1.0"] - states the diversity of sampled sequences with higher numbers resulting in more diverse sequences

**rm_aa** ['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y'] - allows you to remove certain amino acids

**use_solubleMPNN** ["True", "False"] - there sadly where some issues in Jupyter Lab when trying to include this, so you have to remove the arguments (cursive) during initialisation of the mpnn model in 
RFdiffusionJupyter/RFdiffusion/params/ColabDesign/colabdesign/rf/designability_test.py
print("running proteinMPNN...")
  sampling_temp = 0.1
  mpnn_model = mk_mpnn_model(*weights="soluble" if o.use_soluble else "original"*)
When you want to use the soluble weights, you must then cd into: RFdiffusionJupyter/RFdiffusion/params/ColabDesign/colabdesign/mpnn/model.py and change the standard for the 'weights' variable to either 'original' or 'soluble'within the __init__ method of the class.

class mk_mpnn_model():
  def __init__(self, model_name="v_48_020",
               backbone_noise=0.0, dropout=0.0,
               seed=None, verbose=False, weights="original"): # weights can be set to either original or soluble
'

***AF2 settings:***

**initial_guess** ["True", "False"] and **num_recycles** ["0", "1", "2", "3", "6", "12"] - for **binder** design, it is recommended to choose `initial_guess=True num_recycles=3´

**use_multimer** ["True", "False"] - use AlphaFold Multimer v3 params for prediction

In [ ]:
start = time.time()
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
num_seqs = 8 
num_recycles = 3
mpnn_sampling_temp = 0.1 
rm_aa = "C" 
use_solubleMPNN = False
initial_guess = True
use_multimer = False 
os.chdir(os.path.expanduser("~"))
if not os.path.isfile("RFdiffusionJupyter/RFdiffusion/params/done.txt"):
  print("downloading AlphaFold params...")
  while not os.path.isfile("params/done.txt"):
    time.sleep(5)

contigs_str = ":".join(contigs)
opts = [f"--pdb=../outputs/{path}_0.pdb",
        f"--loc=../outputs/{path}",
        f"--contig={contigs_str}",
        f"--copies={copies}",
        f"--num_seqs={num_seqs}",
        f"--num_recycles={num_recycles}",
        f"--rm_aa={rm_aa}",
        f"--mpnn_sampling_temp={mpnn_sampling_temp}",
        f"--num_designs={num_designs}"]
if initial_guess: opts.append("--initial_guess")
if use_multimer: opts.append("--use_multimer")
if use_solubleMPNN: opts.append("--use_soluble")
opts = ' '.join(opts)
os.chdir(os.path.expanduser("~/RFdiffusionJupyter/RFdiffusion/params"))
!python ColabDesign/colabdesign/rf/designability_test.py {opts}
end = time.time()
print(f"Sequence design and ranking took {end - start} seconds")

With this final cell you can display the best sequence for each design. If you want to specify of which design you want to see the best sequence, change the design variable to to according design number.

In [ ]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
import py3Dmol
os.chdir(os.path.expanduser("~"))
design = "best"
def plot_pdb(num = "best"):
  if num == "best":
    with open(f"RFdiffusionJupyter/RFdiffusion/outputs/{path}/best.pdb","r") as f:
      # REMARK 001 design {m} N {n} RMSD {rmsd}
      info = f.readline().strip('\n').split()
    num = info[3]
  hbondCutoff = 4.0
  view = py3Dmol.view(js='https://3dmol.org/build/3Dmol.js')
  pdb_str = open(f"RFdiffusionJupyter/RFdiffusion/outputs/{path}_{num}.pdb",'r').read()
  view.addModel(pdb_str,'pdb',{'hbondCutoff':hbondCutoff})
  pdb_str = open(f"RFdiffusionJupyter/RFdiffusion/outputs/{path}/best_design{num}.pdb",'r').read()
  view.addModel(pdb_str,'pdb',{'hbondCutoff':hbondCutoff})

  view.setStyle({"model":0},{'cartoon':{}}) #: {'colorscheme': {'prop':'b','gradient': 'roygb','min':0,'max':100}}})
  view.setStyle({"model":1},{'cartoon':{'colorscheme': {'prop':'b','gradient': 'roygb','min':0,'max':100}}})
  view.zoomTo()
  view.show()

if num_designs>1:
    plot_pdb(design)
else:
    plot_pdb(num)
